In [100]:
# Imports
import os
import pandas as pd
import requests
from dotenv import load_dotenv
from pathlib import Path

In [101]:
#Read the FMP API Key
load_dotenv()
#Retrive environment variable and store in Python variable
api_key= os.getenv('FMP_API_KEY')
#confirm retrieval of api key
type(api_key)

str

## Define Variables

In [102]:
crypto_symbol = ['BTCUSD','LTCUSD','XRPUSD','ETHUSD','ETCUSD']
url = (f'https://financialmodelingprep.com/api/v3/historical-price-full/crypto/{crypto_symbol}?apikey={api_key}')

In [103]:
def get_crypto_data(cryptourl):
    #get companies based on criteria defined about
    CryptoUSD = requests.get(url)
    CryptoUSD = CryptoUSD.json()
    CryptoUSD = CryptoUSD['historical']
    Crypto = pd.DataFrame.from_dict(CryptoUSD)
    Crypto.set_index('date',inplace=True)    
    #Keep only the close column- Open, close, adjclose, volume,chanepercent
    columns =['change','high','label','low','unadjustedVolume','vwap','changeOverTime','close']
    Crypto= Crypto.drop(columns, axis=1)
    #convert volumne into float datatype
    #Crypto['volume'] = Crypto['volume'].astype(float)
    #Crypto['volume'] = pd.to_numeric(Crypto['volume'], errors='coerce')
    #Rename the column adjclose and changepercent to close and percentchange
    Crypto.rename(columns = {'adjClose' : 'close' , 'changePercent' : 'change%'}, inplace = True)
    return Crypto

In [104]:
def saveto_csv(dataFrame,fileName):    
    newfilename= "Data\\"  + fileName +".csv"    
    dataFrame.to_csv(newfilename,index = True)

In [105]:
for symbol in crypto_symbol:
    url = (f'https://financialmodelingprep.com/api/v3/historical-price-full/crypto/{symbol}?apikey={api_key}')
    crypto_df= get_crypto_data(url)
    saveto_csv(crypto_df,symbol)

## Calculate returns of crypto data weightage is 0.02

In [144]:
def get_crypto_returns(csvFile):   
    # set the file path
    file_path = Path(csvFile)
    # create a Pandas DataFrame from a csv file
    df = pd.read_csv(file_path)   
    # drop null values
    df = df.dropna().copy()   
    #set date as index
    df = df.set_index("date")
    # drop all columns except close
    columns =['open','volume','change%']
    df = df.drop(columns, axis=1)
    #rename column
    columnname = symbol[:-3]
    df.rename(columns = {'close' : columnname}, inplace = True)   
    return df    

In [146]:
for symbol in crypto_symbol: 
    csvFile = 'Data\\'+symbol+'.csv'
    if symbol == 'BTCUSD':
        btc_df=get_crypto_returns(csvFile)  
    elif symbol == 'LTCUSD':
        ltc_df=get_crypto_returns(csvFile)
    elif symbol == 'XRPUSD':
        xrp_df=get_crypto_returns(csvFile)
    elif symbol == 'ETCUSD':
        etc_df=get_crypto_returns(csvFile)
    else:
        eth_df=get_crypto_returns(csvFile)

combine_crypto_df = pd.concat([btc_df,ltc_df,xrp_df,etc_df,eth_df], axis = 'columns', join='inner')
saveto_csv(combine_crypto_df,'CrytoAll')